# Import Library Files

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from tqdm import tqdm

----
1. DEFT Module Components
----

In [2]:
class LocalizationNetwork(nn.Module):
    def __init__(self, input_channels):
        super(LocalizationNetwork, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, kernel_size=5)

        # Placeholder fc1 (will update after shape calculation)
        self.fc1 = nn.Linear(1, 32) 
        self.fc2 = nn.Linear(32, 6)  # 6 affine parameters

        # Initialize weights for identity transformation
        self.fc2.weight.data.zero_()
        self.fc2.bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically compute the flattened size
        if not hasattr(self, 'computed_fc1'):
            flattened_size = x.view(x.shape[0], -1).shape[1]
            self.fc1 = nn.Linear(flattened_size, 32).to(x.device)
            self.computed_fc1 = True  # Prevent re-initialization

        x = x.view(x.shape[0], -1)  # Flatten
        x = F.relu(self.fc1(x))
        theta = self.fc2(x)
        theta = theta.view(-1, 2, 3)
        return theta


class WeightingModule(nn.Module):
    def __init__(self, sigma=0.5):
        super(WeightingModule, self).__init__()
        self.lambda_param = nn.Parameter(torch.tensor(0.5))
        self.sigma = sigma

    def forward(self, grid):
        dist2 = grid[..., 0] ** 2 + grid[..., 1] ** 2
        weight = 1 + self.lambda_param * torch.exp(-dist2 / (2 * self.sigma ** 2))
        return weight.unsqueeze(-1)


class DEFTModule(nn.Module):
    def __init__(self, input_channels, sigma=0.5):
        super(DEFTModule, self).__init__()
        self.localization = LocalizationNetwork(input_channels)
        self.weighting = WeightingModule(sigma)

    def forward(self, x):
        theta = self.localization(x)
        grid = F.affine_grid(theta, x.size(), align_corners=False)
        weight = self.weighting(grid)
        x_transformed = F.grid_sample(x, grid, align_corners=False)

        if x.shape[1] > 1:
            weight = weight.expand(-1, x.shape[2], x.shape[3], x.shape[1]).permute(0, 3, 1, 2)
        else:
            weight = weight.permute(0, 3, 1, 2)

        x_weighted = x_transformed * weight
        return x_weighted

----
2. ResNet50 Model for Feature Extraction
----

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = resnet50(weights=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])  # Remove classification layer
resnet_model.eval().to(device)

# Load DEFT Module
deft_model = DEFTModule(input_channels=3).to(device)  # Input channels = 3 (RGB)
deft_model.eval()

###########################################################
# Define Image Transformation
###########################################################

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

----
3. Feature Extraction Function with DEFT
----

In [5]:
def extract_features(image_path, transform, device=device):
    """Extract ResNet50 features from a single image after DEFT transformation."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)  # Convert to tensor and send to GPU

        # Apply DEFT Transformation
        with torch.no_grad():
            image = deft_model(image)  # Pass through DEFT Module
            features = resnet_model(image).squeeze().cpu().numpy()  # Extract ResNet50 features
        
        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None


###########################################################
# Paths to RGB and Optical Flow Directories
###########################################################

rgb_path = "D:/Datasets/Datasets/EPIC_Kitchen/RGB/P01_04"
flow_u_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u"
flow_v_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v"

label_csv_path = "D:/Datasets/Datasets/EPIC_Kitchen/Label/P01_04.csv"
labels_df = pd.read_csv(label_csv_path)

output_csv = "../Features/Features_DEFT/Feature_P01_04_DEFT.csv"


# -------------------------------------------------------------
# Matching Frames Between RGB and Optical Flow
# -------------------------------------------------------------
def find_closest_frame(frame_name, flow_frames):
    frame_num = int(frame_name.split('_')[1].split('.')[0])  # Extract frame number
    flow_nums = [int(f.split('_')[1].split('.')[0]) for f in flow_frames]
    closest_frame = flow_frames[np.argmin([abs(frame_num - num) for num in flow_nums])]
    return closest_frame

---
4. Extract Features with DEFT & Save
----

In [6]:
S = 5  # Sampling every 30th frame
features_list = []

all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting Features with DEFT"):
    rgb_frame_path = os.path.join(rgb_path, frame)
    flow_u_frame_path = os.path.join(flow_u_path, frame)
    flow_v_frame_path = os.path.join(flow_v_path, frame)
    
    rgb_features = extract_features(rgb_frame_path, transform, device)
    flow_u_features = extract_features(flow_u_frame_path, transform, device)
    flow_v_features = extract_features(flow_v_frame_path, transform, device)
    
    if rgb_features is not None and flow_u_features is not None and flow_v_features is not None:
        combined_features = np.concatenate([rgb_features, flow_u_features, flow_v_features])
        
        frame_number = int(frame.split('_')[-1].split('.')[0])
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]
        
        if not label_row.empty:
            verb_class = label_row.iloc[0]['Verb_class']
            noun_class = label_row.iloc[0]['Noun_class']
            action_class = label_row.iloc[0]['Action_class']
        else:
            verb_class, noun_class, action_class = -1, -1, -1  # Default if no label found
        
        features_list.append([frame, verb_class, noun_class, action_class] + combined_features.tolist())

if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

columns = ["Frame", "Verb_class", "Noun_class", "Action_class"] + [f"Feature_{i}" for i in range(len(combined_features))]
df = pd.DataFrame(features_list, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Feature extraction completed! Saved to {output_csv}")


Extracting Features with DEFT:  50%|████████████████████████▏                       | 636/1262 [00:53<00:39, 15.96it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_03160.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_03160.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_03160.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_03160.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_03165.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_03165.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_03165.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_03165.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_03170.jpg due to error: [Errno 2] No such file or d

Extracting Features with DEFT:  51%|████████████████████████▍                       | 642/1262 [00:53<00:31, 19.92it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_03190.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_03190.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_03190.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_03190.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_03195.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_03195.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_03195.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_03195.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_03200.jpg due to error: [Errno 2] No such file or d

Extracting Features with DEFT:  51%|████████████████████████▋                       | 648/1262 [00:53<00:28, 21.89it/s]

...

Extracting Features with DEFT: 100%|██████████████████████████████████████████████▊| 1258/1262 [01:22<00:00, 21.98it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_06275.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_06275.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_06275.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_06275.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_06280.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_06280.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_06280.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_06280.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_06285.jpg due to error: [Errno 2] No such file or d

Extracting Features with DEFT: 100%|███████████████████████████████████████████████| 1262/1262 [01:22<00:00, 15.23it/s]

Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_06300.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_06300.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_06300.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_06300.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/u\frame_06305.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\u\\frame_06305.jpg'
Skipping frame: D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_04/v\frame_06305.jpg due to error: [Errno 2] No such file or directory: 'D:\\Datasets\\Datasets\\EPIC_Kitchen\\OpticalFlow\\P01_04\\v\\frame_06305.jpg'


Feature extraction completed! Saved to ../Features/Features_DEFT/Feature_P01_04_DEFT.csv


In [7]:
###########################################################
# Display Extracted Features for Verification
###########################################################
f = pd.read_csv("../Features/Features_DEFT/Feature_P01_04_DEFT.csv")
f

,Frame,Verb_class,Noun_class,Action_class,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,...,Feature_6134,Feature_6135,Feature_6136,Feature_6137,Feature_6138,Feature_6139,Feature_6140,Feature_6141,Feature_6142,Feature_6143
0,frame_00000.jpg,-1,-1,-1,0.651917,0.407176,0.076979,0.357042,1.976730,0.638776,...,0.119569,0.000009,0.029647,0.155487,0.006345,0.000375,0.108428,0.139510,0.029922,0.226609
1,frame_00005.jpg,-1,-1,-1,0.546664,0.483001,0.135680,0.225158,2.126921,0.551493,...,0.157487,0.000000,0.006592,0.087250,0.241418,0.000000,0.131806,0.069871,0.019073,0.259733
2,frame_00010.jpg,0,23,0,0.483140,0.646575,0.202512,0.355270,2.394428,0.560053,...,0.212274,0.019422,0.061624,0.105661,0.028055,0.000000,0.000000,0.177602,0.010195,0.164882
3,frame_00015.jpg,0,23,0,0.404216,0.568184,0.049507,0.525459,2.029208,0.432885,...,0.484715,0.122950,0.030248,0.096550,0.606497,0.011370,0.219069,0.188862,0.002258,0.156196
4,frame_00020.jpg,0,23,0,0.504452,0.529196,0.116372,0.598458,1.932839,0.722018,...,0.611212,0.257860,0.085634,0.485791,0.232887,0.000439,0.003415,0.046124,0.019902,0.666137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,frame_03135.jpg,4,23,14,1.126213,0.175981,1.183292,1.671395,0.540780,0.675402,...,0.249193,0.011448,0.177856,0.290418,0.058419,0.020178,0.000000,0.038382,0.000758,0.110715
628,frame_03140.jpg,4,23,14,0.875532,0.130069,1.162229,1.753934,0.414319,0.467606,...,0.014190,0.004053,0.060821,0.077685,0.000000,0.007218,0.001084,0.041424,0.011284,0.098705
629,frame_03145.jpg,-1,-1,-1,0.990685,0.336301,0.916479,1.528714,0.513048,0.428346,...,0.383098,0.005483,0.029652,0.194086,0.161036,0.001043,0.000000,0.023354,0.198656,0.131415
630,frame_03150.jpg,4,7,15,1.255465,0.304995,1.173652,1.445411,0.552218,0.506052,...,0.182186,0.000100,0.025416,0.243660,0.004754,0.002198,0.000000,0.130777,0.013741,0.037159
